# Visualizar o impacto da PEC 3/2022 sobre as propriedades próximas às praias

Esta controversa [Proposta de Emenda à Constituição](https://legis.senado.leg.br/sdleg-getter/documento?dm=9074970&ts=1717625029758&disposition=inline) foi batizada pela imprensa de __PEC das Praias__, devido a interpretação de que permitiria a "privatização das praias". No entanto, ao afastar-se das polêmicas e interesses desse projeto de legislação, especialmente ao entender do que se tratam os denominados Terrenos da Marinha, vamos ver no mapa quais são esses terrenos que fariam jus ao apelido que botaram na PEC. 

## Carregar os módulos necessários

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import folium
from folium.plugins import MarkerCluster

print(np)
print(pd)
print(sns)
print(plt)
print(folium)
print(MarkerCluster)

<module 'numpy' from '/home/chico_linux/.local/lib/python3.10/site-packages/numpy/__init__.py'>
<module 'pandas' from '/home/chico_linux/.local/lib/python3.10/site-packages/pandas/__init__.py'>
<module 'seaborn' from '/home/chico_linux/.local/lib/python3.10/site-packages/seaborn/__init__.py'>
<module 'matplotlib.pyplot' from '/home/chico_linux/.local/lib/python3.10/site-packages/matplotlib/pyplot.py'>
<module 'folium' from '/home/chico_linux/.local/lib/python3.10/site-packages/folium/__init__.py'>
<class 'folium.plugins.marker_cluster.MarkerCluster'>


## Carregar a base de dados necessária

Os dados foram extraídos da Transparência Ativa SPU (Secretaria do Patrimônio da União) neste site: https://clusterqap2.economia.gov.br/extensions/SPU-Transparencia_Ativa/SPU-Transparencia_Ativa.html

Na interface de dashboard, é possível manusear a *Conceituação de Terreno* e filtrar por `Terreno de Marinha`, e assim, abaixo extrair em Excel os Dados dos Imóveis da União que corresponde ao filtro. 

Foi convertido em .csv para facilitar o carregamento, mas é necessário fazer o tratamento dos dados. 

In [2]:
marine_df = pd.read_csv('src/data/terreno_marinha.csv', 
                        index_col=False,
                        header=0,
                        names=['class', 'rip_id', 'rip_use', 'register_date', 'state', 'city', 'address', 'neighborhood', 'lat', 'long', 'precision_level', 'concept', 'type', 'status', 'owner', 'use_date', 'area', 'owner_area'],
                        sep= ';', 
                        on_bad_lines = 'skip'
                        )

marine_df['register_date'] = pd.to_datetime(marine_df['register_date'], format='%d/%m/%Y', errors='coerce')
marine_df['use_date'] = pd.to_datetime(marine_df['use_date'], format='%d/%m/%Y', errors='coerce')

marine_df = marine_df[(marine_df['lat'] != '-')] # Remover valores nulos
marine_df['lat'] = marine_df['lat'].astype(float)
marine_df['long'] = marine_df['long'].astype(float)

marine_df['area'] = marine_df['area'].str.replace(',', '').astype(float)
marine_df['owner_area'] = marine_df['owner_area'].str.replace(',', '').astype(float)

marine_df.shape

(360711, 18)

São mais de 360 mil imóveis categorizados como "Terrenos da Marinha". 

## Terrenos da Marinha

Vamos ver do que se trata esse terrenos, pois de acordo com a legislação (Decreto-Lei nº 9.760, de 1946), os [Terrenos da Marinha não são aquilo que o nome sugere](https://www.gov.br/gestao/pt-br/assuntos/patrimonio-da-uniao/bens-da-uniao/terrenos-de-marinha): 

>        São terrenos de marinha, em uma profundidade de 33 (trinta e três) metros, medidos horizontalmente, para a parte da terra, da posição da linha do preamar-médio de 1831:

>        a) os situados no continente, na costa marítima e nas margens dos rios e lagoas, até onde se faça sentir a influência das marés;

>        b) os que contornam as ilhas situadas em zona onde se faça sentir a influência das marés.

>        Parágrafo único. Para os efeitos dêste artigo a influência das marés é caracterizada pela oscilação periódica de 5 (cinco) centímetros pelo menos, do nível das águas, que ocorra em qualquer época do ano.

Quando vemos os dados, percebemos que os tais "Terrenos da Marinha" são muitas coisas. Não são só terrenos, vão desde cais, porto, farol até reserva, escola, fazenda, etc. O traço comum que caracteriza eles, fora o que está na lei, é que a maioria pertence à União.

In [3]:
marine_df.groupby(['type'], as_index=False).agg({'rip_id': 'count'}).sort_values(['rip_id'], ascending=False)

,type,rip_id
15,Dominial,358594
50,Terreno,590
24,Farol (farolete),244
18,Espelho D'Água,152
31,Ilha (ilhota),118
16,Edifício / Prédio,98
3,Apartamento,93
36,Outros,89
9,Casa,87
51,Universidade,67


Logo, é importante pontuar que **não há um Terreno da Marinha que seja uma praia**, principalmente pelo que descreve a lei!!!

Dessa forma, fica complicado achar que as praias irão virar propriedade privada, conforme alguns comentários na web...


Mas... vamos fazer um exercício que mostre qual é o potencial mínimo que aborda a tal PEC. Vamos visualizar propriedades que estão próximas às praias, a fim de entender se tal PEC faz um pouco de jus ao apelido dela. 

## Começar o EDA e filtragem

Vamos começar a filtragem. 

Como nós queremos aquilo que **se aproxime mais das praias**, e não há um tipo de Terreno de Marinha que seja conceitualizado como "praia" ou tenha uma classificação "próxima da praia", vamos atrás de elementos que poderiam ser englobados como próximos às praias. 

In [4]:
marine_df = marine_df[marine_df.type.isin(['Terreno', 'Gleba', 'Reserva', 'Ilha (ilhota)'])]

Pensou-se em colocar `Espelho D'Água`, mas fica bem claro que há muitas propriedade fluviais nesse tipo de Terreno. 

Vamos excluir as propriedade que estão em estados que não possuem litoral. 

In [5]:
marine_df = marine_df[~(marine_df.state.isin(['DF', 'MT', 'MG', 'AC', "RN", 'RO', 'AM']))]

Verifique a conceituação do terreno. 

In [6]:
marine_df.groupby(['concept'], as_index=False).agg({'rip_id': 'count'}).sort_values(['rip_id'], ascending=False)

,concept,rip_id
0,Marinha,353
1,Marinha com Acrescido,219
2,Marinha com Nacional Interior,142


Não há muito o que utilizar

Há algumas coordenadas que fogem muito do litoral - exclua elas. 

In [7]:
marine_df = marine_df[~(marine_df.lat == -5.092011) & ~(marine_df.long == -42.80376)]
marine_df = marine_df[~(marine_df.lat == -5.366336945) & ~(marine_df.long == -49.12535756)]
marine_df = marine_df[~(marine_df.lat == -5.349939449) & ~(marine_df.long == -49.13003707)]
marine_df = marine_df[~(marine_df.lat == -28.62924658) & ~(marine_df.long == -56.0263081)]
marine_df = marine_df[~(marine_df.lat == -27.5723604) & ~(marine_df.long == -54.6710219)]
marine_df = marine_df[~(marine_df.lat == -22.774078) & ~(marine_df.long == -53.303135)]
marine_df = marine_df[~(marine_df.lat == -22.41771102) & ~(marine_df.long == -50.57502853)]
marine_df = marine_df[~(marine_df.lat == -22.59499466) & ~(marine_df.long == -48.79830363)]
marine_df = marine_df[~(marine_df.lat == -22.498636) & ~(marine_df.long == -48.557842)]
marine_df = marine_df[~(marine_df.lat == -22.49134) & ~(marine_df.long == -48.558807)]
marine_df = marine_df[~(marine_df.lat == -22.81175045) & ~(marine_df.long == -45.18986504)]
marine_df = marine_df[~(marine_df.lat == -9.405954711) & ~(marine_df.long == -40.50411564)]
marine_df = marine_df[~(marine_df.lat == -8.319732711) & ~(marine_df.long == -38.74383868)]
marine_df = marine_df[~(marine_df.lat == -5.097750713) & ~(marine_df.long == -38.37050173)]

marine_df = marine_df[~(marine_df.lat == -21.762669) & ~(marine_df.long == -41.31845)]
marine_df = marine_df[~(marine_df.lat == -22.09933796) & ~(marine_df.long == -41.14216741)]
marine_df = marine_df[~(marine_df.lat == -22.45539463) & ~(marine_df.long == -43.84043631)]
marine_df = marine_df[~(marine_df.lat == -26.89359229) & ~(marine_df.long == -49.06980476)]
marine_df = marine_df[~(marine_df.lat == -26.90094) & ~(marine_df.long == -49.046413)]
marine_df = marine_df[~(marine_df.lat == -26.934095) & ~(marine_df.long == -48.953907)]

Veja quantas propriedades

In [8]:
marine_df.shape

(692, 18)

Chegamos no número de __700 propriedades que estão SUPOSTAMENTE próximas às praias__. 

Não é _per se_ registros duplicados, mas não faz sentido para a análise ter propriedades com a mesma coordenada e área, por mais que o RIP do Imóvel (Registro Imobiliário Patrimonial, que foi renomeado para `rip_id` aqui) seja distinto. 

In [9]:
marine_df = marine_df.drop_duplicates(subset=['area', 'lat', 'long'])

marine_df.shape

(632, 18)

Crie variáveis irão ajudar a plotar informações no mapa para posterior análise. 

In [10]:
marine_df['tooltip'] = marine_df['type'] + ' - ' + marine_df['rip_id'].astype(str)

In [11]:
marine_df['color'] = np.where(marine_df['type'] == 'Terreno', '#2874A6', '#5499C7')
marine_df['color'] = np.where(marine_df['type'] == 'Gleba', '#85C1E9', marine_df['color'])
marine_df['color'] = np.where(marine_df['type'] == 'Reserva', '#1F618D', marine_df['color'])

## Construção do mapa

Hora de criar o mapa - vá pelo caminho mais simples. 

In [12]:
my_map = folium.Map(
                    location = [-11.733350280370129, -49.05221944693883], # Centralizar no Brasil
                    zoom_start = 5,
                    tiles = 'OpenStreetMap'
                    )

lat = marine_df[marine_df.lat.notnull()].lat.tolist()
long = marine_df[marine_df.long.notnull()].long.tolist()
marine_info = marine_df.tooltip.tolist()
marine_color = marine_df.color.tolist()

for x, y, z, w in zip(lat, long, marine_info, marine_color):

    folium.CircleMarker(location=[x, y], 
                        fill=True, 
                        popup=z,
                        color=w).add_to(my_map)
    
folium.plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(my_map)


In [13]:
my_map

Ficou um mapa aceitável para entender as possibilidades da PEC

Vamos desenhar um mapa que seja mais elegante - faça agregações

https://python-visualization.github.io/folium/latest/user_guide/plugins/marker_cluster.html

In [14]:
my_map = folium.Map(
                    location = [-11.733350280370129, -49.05221944693883], # Centralizar no Brasil
                    zoom_start = 5,
                    tiles = 'Cartodb Positron'
                    )

locations = list(zip(lat, long))

MarkerCluster(locations=locations, 
                    popups=locations,
                    ).add_to(my_map)

In [15]:
my_map

Ficou boa a visualização e dá para entender o potencial da PEC ao envolver propriedade mais próximas de praias. 